<a href="https://colab.research.google.com/github/gnswp21/uda/blob/main/data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torch
from torchtext import data, datasets
import random

In [4]:
SEED = 5
random.seed(SEED)
torch.manual_seed(SEED)

In [ ]:
!pip install fastBPE regex requests sacremoses subword_nmt
!pip install omegaconf
!pip install fairseq

In [ ]:
# Round-trip translations between English and German:
en2de = torch.hub.load('pytorch/fairseq', 'transformer.wmt19.en-de.single_model', tokenizer='moses', bpe='fastbpe')
de2en = torch.hub.load('pytorch/fairseq', 'transformer.wmt19.de-en.single_model', tokenizer='moses', bpe='fastbpe')

paraphrase = de2en.translate(en2de.translate('PyTorch Hub is an awesome interface!'))
assert paraphrase == 'PyTorch Hub is a fantastic interface!'

In [5]:
#train_iter, test_iter = datasets.IMDB(root='data', split=('train', 'test'))
train_iter = datasets.IMDB(split=('train'))

aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:01<00:00, 44.4MB/s]


In [6]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

tokenizer = get_tokenizer('basic_english')

def yield_tokens(data_iter):
    for _, text in data_iter:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])


In [7]:
text_pipeline = lambda x: vocab(tokenizer(x))
def text_pipeline(x):
  x = vocab(tokenizer(x))
  if len(x) >= 500:
    return x[:500]
  else:
    zeros = [0 for i in range(500)]
    zeros[:len(x)] = x[:]
    return zeros

label_pipeline = lambda x: 0 if x == 'neg' else 1

In [8]:
from torch.utils.data import DataLoader
from torchtext.datasets import AG_NEWS
from torch.nn.utils.rnn import pad_sequence

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def collate_batch(batch):
    label_list, text_list = [], []
    for (_label, _text) in batch:
         label_list.append(label_pipeline(_label))
         processed_text = torch.tensor(text_pipeline(_text),dtype=torch.int64)
         text_list.append(processed_text)
    label_list = torch.tensor(label_list, dtype=torch.int64)
    text_list = pad_sequence(text_list, batch_first=True, padding_value=0)
    return label_list.to(device), text_list.to(device)


train_loader = DataLoader(train_iter, batch_size=30,collate_fn=collate_batch)